# Numerical Verification for the Uniform Weight Model

This notebook provides a computer-assisted verification of numerical bounds used in the drift analysis of the $(1+1)$-EA on noisy linear functions with weights i.i.d. $\mathrm{Unif}(0,1)$.

## What it verifies

1. For each $k \in \{0,\dots,9\}$, we compute an exact polynomial $H''_k(\alpha)$ (second derivative w.r.t. $\alpha$) and a lower bound $m_k \le H''_k(\alpha)$ for all $\alpha\in[0,1]$.
2. Using a tail bound for $k \ge 10$, we build a polynomial $P(\chi)$ (as in the thesis) and certify:
   - $P(\chi_{\max})>0$ for $\chi_{\max}=14/5$,
   - no real root of $P$ lies in $(0,\chi_{\max}]$.

This implies $P(\chi)\ge 0$ for all $\chi \in (0,\chi_{\max}]$, by continuity.

## How to run

Run this notebook in a SageMath environment. If you use Jupyter, select a SageMath kernel, or run Jupyter from Sage (e.g. `sage -n jupyter`).


In [91]:
def sage_version():
    try:
        from sage.env import SAGE_VERSION
        return str(SAGE_VERSION)
    except Exception:
        import sage
        return str(
            getattr(getattr(sage, "version", None), "version", None)
            or getattr(sage, "__version__", "unknown")
        )

print("Sage version:", sage_version())


Sage version: 10.8


## Mutation Parameters

We work on the interval $\chi \in (0,\chi_{\max}]$ with $\chi_0<\chi_{\max}<11$.


In [92]:
CHI_MAX = QQ(14)/5
assert CHI_MAX < 11


## Acceptance Probabilities $p_A(k,i)$

For the uniform weights case, the acceptance probabilities are expressed via the Irwin-Hall CDF:
$$
p_A(k,i) = \frac{1}{k!}\sum_{m = 0}^i (-1)^m\binom{k}{m}(i - m)^k.
$$

In [93]:
def irwin_hall_cdf(k, i):
    '''
    Irwin–Hall CDF F_k(i) at an integer i, for the sum of k i.i.d. Unif(0,1).
    Returns an exact rational in QQ.
    '''
    k = Integer(k)
    i = Integer(i)
    if k == 0:
        return QQ(1) if i >= 0 else QQ(0)
    if i <= 0:
        return QQ(0)
    if i >= k:
        return QQ(1)

    s = ZZ(0)
    for m in range(i + 1):
        s += (-1)**m * binomial(k, m) * (i - m)**k
    return QQ(s) / factorial(k)


## Constructing $H_k(\alpha)$ and $H''_k(\alpha)$ exactly for $k\le 9$

We compute
$$
H_k(\alpha) = \sum_{i=0}^k (2i-k)\,p_A(k,i)\,\binom{k}{i}\alpha^i(1-\alpha)^{k-i},
$$
as a polynomial in $\alpha$ over $\mathbb{Q}$, and then take the second derivative.


In [94]:
R.<a> = PolynomialRing(QQ)

Delta_poly = []
Delta_dd = []

for k in range(10):
    s = R(0)
    for i in range(k + 1):
        pA = R(irwin_hall_cdf(k, i))
        s += (2*i - k) * pA * binomial(k, i) * a**i * (1 - a)**(k - i)
    s = R(s)
    Delta_poly.append(s)
    Delta_dd.append(s.derivative(2))

# check if polynomials in QQ[a]
assert all(p.parent() == R for p in Delta_dd)


## Lower Bounds $m_k \le H''_k(\alpha)$ for all $\alpha\in[0,1]$

For a univariate polynomial, the minimum on $[0,1]$ is attained at an endpoint or a critical point.
We compute critical points in the algebraic reals (AA), evaluate exactly there, and take the minimum.
For the two cases where the minimum is irrational (here: $k=8,9$), we round down to a rational
using interval arithmetic.


In [95]:
from math import floor


def compute_mk_AA(poly):
    q = poly.change_ring(AA)
    dq = q.derivative()
    cand = [AA(0), AA(1)]
    if not dq.is_zero():
        for r, mult in dq.roots(ring=AA):
            if (r > 0) and (r < 1):
                cand.append(r)
    vals = [q(c) for c in cand]
    return min(vals)

def certified_lower_rational(x, d=12, bits=800):
    '''
    Return a rational <= x by taking a certified floor on a 10^{-d} grid.
    '''
    xAA = AA(x)                     # case to AA
    if xAA in QQ:                   # if rational, return as is
        return QQ(xAA)

    scale = ZZ(10)**d               # scale to 10^{-d}
    RIF = RealIntervalField(bits)   # interval containing xAA
    I = RIF(xAA) * scale            # scale up to 10^{-d}
    n = floor(I.lower())            # take floor of lower bound of interval
    return QQ(n) / scale            # scale back down to 10^{-d}


In [96]:
mk = [None]*10

for k in range(8):
    m = compute_mk_AA(Delta_dd[k])
    assert m in QQ
    mk[k] = QQ(m)
    assert mk[k] == m

for k in [8, 9]:
    m = compute_mk_AA(Delta_dd[k])
    mk[k] = certified_lower_rational(m, d=12, bits=800)
    assert mk[k] <= m

for k in range(10):
    print(f"m_{k} =", mk[k])


m_0 = 0
m_1 = 0
m_2 = 4
m_3 = 7
m_4 = 2
m_5 = -7/2
m_6 = -9/2
m_7 = -685757/229680
m_8 = -2412895665423/1000000000000
m_9 = -2283273660679/1000000000000


## Tail Bound for $k\ge 10$

Let $r\coloneqq\chi/11$. For $\vert\chi\vert<11$ (equivalently $\vert r\vert<1$) the series converges absolutely and we can use the standard power-sum identities obtained by differentiating the geometric series. Expanding $(j+10)^3$ and combining the closed forms for each series, for all $\vert\chi\vert<11$ we get
$$
\sum_{j\ge 0} (j+10)^3\left(\frac{\chi}{11}\right)^j
= \frac{14641000 - 3552439\chi + 290884\chi^2 - 8019\chi^3}{(11-\chi)^4}.
$$
We evaluate it at $\chi_{\max}=14/5$ and also provide the coarse bound $1504$ used for a cleaner expression.


In [97]:
def tail_series_closed_form(chi):
    '''
    Exact closed form for sum_{j>=0} (j+10)^3 (chi/11)^j, valid for |chi|<11.
    Returned in QQ if chi is rational.
    '''
    chi = QQ(chi)
    if abs(chi) >= 11:
        raise ValueError("require |chi| < 11")
    num = QQ(14641000) - QQ(3552439)*chi + QQ(290884)*chi**2 - QQ(8019)*chi**3
    den = (QQ(11) - chi)**4
    return num / den

TAIL_AT_CHI_MAX = tail_series_closed_form(CHI_MAX)
TAIL_COARSE = QQ(1504)

print("Tail closed form at chi_max:", TAIL_AT_CHI_MAX)
print("Coarse tail bound:", TAIL_COARSE)


Tail closed form at chi_max: 4249167670/2825761
Coarse tail bound: 1504


## Build the polynomial $P(\chi)$ and exclude roots on $(0,\chi_{\max}]$

We form
$$
P(\chi) = \sum_{k=0}^{9} m_k \frac{\chi^k}{k!} - \frac{7}{5} \cdot C \cdot \frac{\chi^{10}}{10!},
$$
where $C$ is either:
- $C = \text{Tail}(\chi_{\max})$ or
- $C = 1504$ (coarser but simpler).


In [98]:
S.<x> = PolynomialRing(QQ)

def build_P(C):
    P = S(0)
    for k in range(10):
        P += S(mk[k]) * x**k / QQ(factorial(k))
    P -= QQ(7)/QQ(5) * QQ(C) * x**10 / QQ(factorial(10))
    return S(P)

P_strong = build_P(TAIL_AT_CHI_MAX)
P_coarse = build_P(TAIL_COARSE)

print("deg(P_strong) =", P_strong.degree())
print("deg(P_coarse) =", P_coarse.degree())


deg(P_strong) = 10
deg(P_coarse) = 10


In [99]:
def assert_no_root_in_open_interval(poly, lo, hi):
    '''
    Assert poly has no real root in (lo, hi] by enumerating algebraic real roots in AA.
    '''
    lo = AA(lo); hi = AA(hi)
    roots = poly.roots(ring=AA)
    bad = []
    for r, mult in roots:
        if (r > lo) and (r <= hi):
            bad.append((r, mult))
    assert len(bad) == 0, f"Found real root(s) in (lo,hi]: {bad}"

def certify_nonnegative_on_interval(poly, hi):
    '''
    Certify poly >= 0 on (0,hi] by:
      - checking poly(hi) > 0
      - asserting no real roots in (0,hi]
    '''
    val = poly(QQ(hi))
    assert val > 0, f"poly(hi) is not > 0, got {val}"
    assert_no_root_in_open_interval(poly, 0, hi)
    return val

val_strong = certify_nonnegative_on_interval(P_strong, CHI_MAX)
val_coarse = certify_nonnegative_on_interval(P_coarse, CHI_MAX)

print("P_strong(chi_max) =", val_strong)
print("P_coarse(chi_max) =", val_coarse)


P_strong(chi_max) = 7168051991042065190757385984883/356517766010742187500000000000
P_coarse(chi_max) = 2536283829418249289891603/126166992187500000000000


## Display Roots


In [100]:
def print_roots(poly, digits=80):
    for r, mult in poly.roots(ring=AA):
        print(r.n(digits), "mult =", mult)

print("Roots of P_strong:")
print_roots(P_strong)

print("\nRoots of P_coarse:")
print_roots(P_coarse)


Roots of P_strong:
-2.0006048459515536910163 mult = 1
0.00000000000000000000000 mult = 2
3.0831203157279731001607 mult = 1

Roots of P_coarse:
-2.0005828925341902983506 mult = 1
0.00000000000000000000000 mult = 2
3.0830514826730141037510 mult = 1


## Export constants (optional)

Exports $m_k$ and coefficients of $P(\chi)$ to a JSON file.


In [101]:
# JSON export

import json, sys, datetime
from pathlib import Path

def poly_to_dict(poly, var="chi"):
    """
    Export univariate polynomial poly in S=QQ[var] as:
      - degree d
      - coeffs: {"d": c_d, ..., "0": c_0} (all included, including zeros)
    Coefficients are stored as QQ strings.
    """
    d = int(poly.degree())
    coeffs = {str(i): str(QQ(poly[i])) for i in range(d, -1, -1)}
    return {"var": var, "degree": d, "coeffs": coeffs}

export = {
    "metadata": {
        "sage_version": sage_version(),
        "python_version": sys.version.split()[0],
        "timestamp_utc": datetime.datetime.utcnow().isoformat(timespec="seconds") + "Z",
    },
    "chi_max": str(CHI_MAX),

    # m_k with explicit indices
    "mk": {str(k): str(QQ(mk[k])) for k in range(len(mk))},

    "tail_at_chi_max": str(TAIL_AT_CHI_MAX),
    "tail_coarse": str(TAIL_COARSE),

    "P_strong": poly_to_dict(P_strong, var="chi"),
    "P_coarse": poly_to_dict(P_coarse, var="chi"),
}

out_path = Path("..") / "results" / "results.json"
out_path.parent.mkdir(parents=True, exist_ok=True)
out_path.write_text(json.dumps(export, indent=2), encoding="utf-8")
print("Wrote", out_path)


Wrote ../results/results.json
